## MLflow Prompt Management Lab 

## Step 1: Install Required Dependencies
Install the necessary packages for MLflow, pandas, scikit-learn, and pyngrok:

In [ ]:
!pip install mlflow pandas scikit-learn pyngrok -q

In [ ]:
!pip install langchain-openai -q

In [ ]:
# Install ngrok
!pip install pyngrok -q

# Authenticate ngrok with your API key
from pyngrok import ngrok
ngrok.set_auth_token("YOUR_API_KEY")  # Replace with your ngrok authtoken

In [ ]:
import subprocess
from pyngrok import ngrok

# Start MLflow server in the background
get_ipython().system_raw("mlflow server --host 127.0.0.1 --port 5000 &")

# Create an ngrok tunnel to the MLflow server
public_url = ngrok.connect(5000, "http")
print(f"MLflow UI is available at: {public_url}")

MLflow UI is available at: NgrokTunnel: "https://59f46c12d836.ngrok-free.app" -> "http://localhost:5000"


## Step 2: Initialize MLflow and Set Up Experiment
Set up MLflow tracking and create an experiment for prompt management:

In [ ]:
import mlflow
import mlflow.tracking
import pandas as pd
from datetime import datetime
import os
import subprocess

# Start MLflow - this will track everything for us
mlflow.set_tracking_uri(" http://127.0.0.1:5000")
mlflow.set_experiment("Prompt Management Lab")

print("✅ MLflow is ready to manage our prompts!")

2025/07/11 04:52:35 INFO mlflow.tracking.fluent: Experiment with name 'Prompt Management Lab' does not exist. Creating a new experiment.


✅ MLflow is ready to manage our prompts!


## Step 3: Register a Prompt Template
Register your first prompt template in MLflow:

In [ ]:
import mlflow

system_prompt = mlflow.genai.register_prompt(
    name="chatbot_prompt",
    template="You are a chatbot that can answer questions about IT. Answer this question: {{question}}",
    commit_message="Initial version of chatbot",
)

2025/07/11 04:52:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: chatbot_prompt, version 1


## Step 4: Create LangChain Integration
Convert the MLflow prompt to LangChain format and build a processing chain:

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

OPENAI_API_KEY = "Your_api_key"

# Convert MLflow prompt to LangChain format
prompt = ChatPromptTemplate.from_template(system_prompt.to_single_brace_format())

# Build the chain: prompt → LLM → output parser
chain = prompt | ChatOpenAI(api_key = OPENAI_API_KEY,temperature=0.7) | StrOutputParser()

# Test the chain
question = "What is MLflow?"
print(chain.invoke({"question": question}))
# MLflow is an open-source platform for mana

MLflow is an open-source platform for managing the end-to-end machine learning lifecycle. It allows data scientists to track experiments, package code into reproducible runs, and share and deploy models. MLflow also supports various machine learning libraries and frameworks, making it easier to work with different tools within the same platform.


## Step 5: Enable Model Tracking and Autologging
Set up automatic tracking for all LLM interactions:

In [ ]:
# Set the active model for linking traces
mlflow.set_active_model(name="langchain_model")

# Enable autologging - all traces will be automatically linked to the active model
mlflow.langchain.autolog()

2025/07/11 04:58:26 INFO mlflow.tracking.fluent: LoggedModel with name 'langchain_model' does not exist, creating one...
2025/07/11 04:58:27 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-5af7da4b6b5c4d80ad2be9eabf4f253a


## Step 6: Run Multiple Test Questions and Track Results
Execute multiple questions and verify trace tracking:

In [ ]:
questions = [
    {"question": "What is MLflow Tracking and how does it work?"},
    {"question": "What is Unity Catalog?"},
    {"question": "What are user-defined functions (UDFs)?"},
]
outputs = []

for question in questions:
    outputs.append(chain.invoke(question))

# Verify traces are linked to the active model
active_model_id = mlflow.get_active_model_id()
mlflow.search_traces(model_id=active_model_id)

,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,c377c37f50b54db798e94721ca10c06a,Trace(trace_id=c377c37f50b54db798e94721ca10c06a),None,TraceState.OK,1752209914529,596,{'question': 'What are user-defined functions ...,User-defined functions (UDFs) are functions cr...,"{'mlflow.traceInputs': '{""question"": ""What are...","{'mlflow.traceName': 'RunnableSequence', 'mlfl...","[{'trace_id': 'pfJVLw91ZYvO/qo3RWuw3g==', 'spa...",[]
1,6817cbbbdeb946458d4725725e003014,Trace(trace_id=6817cbbbdeb946458d4725725e003014),None,TraceState.OK,1752209913943,570,{'question': 'What is Unity Catalog?'},Unity Catalog is a centralized repository of a...,"{'mlflow.traceInputs': '{""question"": ""What is ...","{'mlflow.traceName': 'RunnableSequence', 'mlfl...","[{'trace_id': 'zLeAlCQ8LcyB4ntTZMp1Bw==', 'spa...",[]
2,320efcb6d2cd4a5fa462f2692590f8e8,Trace(trace_id=320efcb6d2cd4a5fa462f2692590f8e8),None,TraceState.OK,1752209912631,1295,{'question': 'What is MLflow Tracking and how ...,MLflow Tracking is a component of the open-sou...,"{'mlflow.traceInputs': '{""question"": ""What is ...","{'mlflow.traceName': 'RunnableSequence', 'mlfl...","[{'trace_id': 'UdX/Jg5J/vIcg9St3am9RA==', 'spa...",[]


[Trace(trace_id=320efcb6d2cd4a5fa462f2692590f8e8), Trace(trace_id=6817cbbbdeb946458d4725725e003014), Trace(trace_id=c377c37f50b54db798e94721ca10c06a)]